In [1]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2

# Paths to dataset directories
train_dir = "C:\\Users\\Pc\\OneDrive\\Desktop\\car_damage\\damage_locate\\train"
val_dir = "C:\\Users\\Pc\\OneDrive\\Desktop\\car_damage\\damage_locate\\val"
test_dir = "C:\\Users\\Pc\\OneDrive\\Desktop\\car_damage\\damage_locate\\test"

# Load annotations
def load_annotations(json_file):
    with open(json_file) as f:
        return json.load(f)

# Prepare the dataset
def load_data(images_dir, annotations_json):
    annotations = load_annotations(annotations_json)
    images = []
    labels = []

    for img_info in annotations['images']:
        img_id = img_info['id']
        img_path = os.path.join(images_dir, img_info['file_name'])
        
        # Load the image
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Unable to load image at {img_path}. Skipping.")
            continue  # Skip this image if it cannot be loaded

        img = cv2.resize(img, (150, 150))  # Resize to your input size
        images.append(img)

        # Create a label array for the parts
        label_array = np.zeros(5)  # Assuming 5 categories
        for ann in annotations['annotations']:
            if ann['image_id'] == img_id:
                part_id = ann['category_id']  # Adjust if needed
                label_array[part_id - 1] = 1  # Assuming category ids are 1-indexed
        labels.append(label_array)

    # Convert lists to NumPy arrays
    images = np.array(images)
    labels = np.array(labels)

    # Ensure consistent input shapes
    if len(images) == 0 or len(labels) == 0:
        raise ValueError("No valid images/labels found.")

    return images, labels

# Load training and validation data
X_train, y_train = load_data(val_dir, os.path.join(val_dir, "COCO_mul_val_annos.json"))
X_val, y_val = load_data(val_dir, os.path.join(val_dir, "COCO_mul_val_annos.json"))

# Normalize the images
X_train = X_train / 255.0
X_val = X_val / 255.0

# Print the shapes of the loaded datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

# Define the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(5, activation='sigmoid')  # Output layer for 5 categories
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Save the model
model.save("car_damage_location_model.h5")
print("Model saved as car_damage_location_model.h5")

# Evaluation on test dataset
X_test, y_test = load_data(val_dir, os.path.join(val_dir, "COCO_mul_val_annos.json"))  # Update with your actual test annotations
X_test = X_test / 255.0

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.2f}")


X_train shape: (11, 150, 150, 3), y_train shape: (11, 5)
X_val shape: (11, 150, 150, 3), y_val shape: (11, 5)
Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 0.6911 - accuracy: 0.0909 - val_loss: 0.7978 - val_accuracy: 0.9091
Epoch 2/10
1/1 [==============================] - 1s 735ms/step - loss: 0.7978 - accuracy: 0.9091 - val_loss: 0.7717 - val_accuracy: 0.9091
Epoch 3/10
1/1 [==============================] - 1s 773ms/step - loss: 0.7717 - accuracy: 0.9091 - val_loss: 0.4994 - val_accuracy: 0.9091
Epoch 4/10
1/1 [==============================] - 1s 801ms/step - loss: 0.4994 - accuracy: 0.9091 - val_loss: 0.4325 - val_accuracy: 0.9091
Epoch 5/10
1/1 [==============================] - 1s 790ms/step - loss: 0.4325 - accuracy: 0.9091 - val_loss: 0.3644 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 1s 764ms/step - loss: 0.3644 - accuracy: 1.0000 - val_loss: 0.2703 - val_accuracy: 0.3636
Epoch 7/10
1/1 [==============================] - 1s 

c:\Users\Pc\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved as car_damage_location_model.h5
1/1 [==============================] - 0s 115ms/step - loss: 0.0294 - accuracy: 0.3636
Test accuracy: 0.36
